# Topics


In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from z3 import *
from tqdm import tqdm

In [2]:
sys.setrecursionlimit(1500)

In [3]:
with open("20-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'#############################################################################################################################################\n#...#...#.....#...#.......#.....#...#...#...#...............#.......#.....#...#...###...#...#.......#...#...#.......#...#.......#...###...###\n#.#.#.#.#.###.#.#.#.#####.#.###.#.#.#.#.#.#.#.#############.#.#####.#.###.#.#.#.#.###.#.#.#.#.#####.#.#.#.#.#.#####.#.#.#.#####.#.#.###.#.###\n#.#.#.#.#...#...#...#.....#.#...#.#.#.#.#.#.#.......#...#...#...#...#...#...#...#...#.#.#.#.#.....#...#.#.#...#.....#.#.#.#.....#.#...#.#...#\n#.#.#.#.###.#########.#####.#.###.#.#.#.#.#.#######.#.#.#.#####.#.#####.###########.#.#.#.#.#####.#####.#.#####.#####.#.#.#.#####.###.#.###.#\n#.#...#...#.........#...#...#...#.#.#.#.#.#.#...###...#.#.#...#.#...#...#...........#.#.#.#.#...#...#...#...#...#...#.#.#.#.#...#.#...#...#.#\n#.#######.#########.###.#.#####.#.#.#.#.#.#.#.#.#######.#.#.#.#.###.#.###.###########.#.#.#.#.#.###.#.#####.#.###.#.#.#.#.#.#.#.#.#.#####.#.#

In [4]:
test_data_raw = r"""###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""






def preprocess_data (data):
    rows = [list(row) for row in data.split("\n")]
    return np.array(rows, dtype='U10')



test_data = preprocess_data(test_data_raw)

display(test_data)


array([['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#'],
       ['#', '.', '.', '.', '#', '.', '.', '.', '#', '.', '.', '.', '.',
        '.', '#'],
       ['#', '.', '#', '.', '#', '.', '#', '.', '#', '.', '#', '#', '#',
        '.', '#'],
       ['#', 'S', '#', '.', '.', '.', '#', '.', '#', '.', '#', '.', '.',
        '.', '#'],
       ['#', '#', '#', '#', '#', '#', '#', '.', '#', '.', '#', '.', '#',
        '#', '#'],
       ['#', '#', '#', '#', '#', '#', '#', '.', '#', '.', '#', '.', '.',
        '.', '#'],
       ['#', '#', '#', '#', '#', '#', '#', '.', '#', '.', '#', '#', '#',
        '.', '#'],
       ['#', '#', '#', '.', '.', 'E', '#', '.', '.', '.', '#', '.', '.',
        '.', '#'],
       ['#', '#', '#', '.', '#', '#', '#', '#', '#', '#', '#', '.', '#',
        '#', '#'],
       ['#', '.', '.', '.', '#', '#', '#', '.', '.', '.', '#', '.', '.',
        '.', '#'],
       ['#', '.', '#', '#', '#', '#', '#', '.', '#', '.', '#', '#', '#',
       

In [5]:
full_data = preprocess_data(data_raw)
full_data

array([['#', '#', '#', ..., '#', '#', '#'],
       ['#', '.', '.', ..., '#', '#', '#'],
       ['#', '.', '#', ..., '#', '#', '#'],
       ...,
       ['#', '.', '#', ..., '#', '.', '#'],
       ['#', '.', '.', ..., '.', '.', '#'],
       ['#', '#', '#', ..., '#', '#', '#']], dtype='<U10')

In [57]:

def solution (data, verbose=False):
    start = tuple(np.argwhere(data == 'S')[0])
    end = tuple(np.argwhere(data == 'E')[0])

    def isValid(shape : tuple, index : tuple):
            return (0 <= index[0] < shape[0] and
                    0 <= index[1] < shape[1])

    def tuple_add (t1, t2):
        return tuple(map(sum, zip(t1, t2)))
    
    # Modified for cheating
    def get_neightbors (array, pos, cheat):
        if not cheat:
            cheat = ((-1,-1), (-1,-1))
        if pos == cheat[0]:
            return [cheat[1]]

        directions = [(1,0), (-1,0), (0,1), (0,-1)]

        next_nodes = [tuple_add(pos, direction) for direction in directions
                                                                if isValid(array.shape, tuple_add(pos, direction)) 
                                                                and (array[tuple_add(pos, direction)] in ['.', 'E'] 
                                                                     or tuple_add(pos, direction) == cheat[0])
                                                                ]
        return next_nodes
    
    # To detect if node is a possible cheat node
    def get_cheat_neightbors (array, pos):
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        next_nodes = [(tuple_add(pos, direction), tuple_add(pos, tuple_add(direction, direction))) for direction in directions
                                                                if isValid(array.shape, tuple_add(pos, direction))
                                                                and  isValid(array.shape, tuple_add(pos, tuple_add(direction, direction)))
                                                                and array[tuple_add(pos, direction)] in ["#"]
                                                                and array[tuple_add(pos, tuple_add(direction, direction))] in [".", "E"] ]
        return next_nodes

    def display_field(field):
        field_list = field.tolist()
        field_str = "\n".join(["".join(row)   for row in field_list])
        return field_str
    
    cheat_blocks = set()
    for start_cheat in np.argwhere(np.isin(data,  ["S", "."])):
        nbs = get_cheat_neightbors(data, start_cheat)
        cheat_blocks.update(set(nbs))

    # field_cheat = data.copy()
    # for cheat in cheat_blocks:
    #     field_cheat[cheat[0]] = 'O'

    
    def path_finding(cheat_block):

        # print(display_field(field))
        # Dijkstra
        field = data.copy()
        # if cheat_block:
        #     field[cheat_block] = '.'
        

        # visited nodes -> cannot be in quere anymore
        visited = []
        # nodes to iterate over
        queue = [start]
        # Tracks lowest costs per node
        costs = defaultdict(lambda: float('inf'))
        costs[start] = 0

        while queue:
            # find smallest
            current_node = queue[0]
            # print(current_node)
            lowest_cost = costs[current_node]
            if len(queue) > 1:
                for node in queue[1:]:
                    if costs[node] < lowest_cost:
                        current_node = node
                        lowest_cost = costs[node]

            queue.remove(current_node)
            visited.append(current_node)

            nbs = get_neightbors(field, current_node, cheat_block)
            # print("Neighbors", nbs)
            for nb in nbs:
                if nb not in visited:
                    if costs[nb] > costs[current_node] +1:
                        costs[nb] = costs[current_node] +1
                        queue.append(nb)

        # if cheat_block:
        #     print(costs[cheat_block[0]], costs[cheat_block[1]])
        return costs[end], costs
    
    
    # print(path_finding(None))
    # print(path_finding(((1,8), (1,9))), "(71)")
    # print(path_finding(((7,10), (7,11))), "(64)")
    # print(path_finding(((8,8), (9,8))), "(46)")
    # print(path_finding(((7,6), (7,5))), "(20)")

    no_cheating, costs_no_cheating = path_finding(None)
    time_saves = []
    for cheat in tqdm(cheat_blocks):
        start_cheat = tuple_add(cheat[0], (cheat[0][0]-cheat[1][0], cheat[0][1]-cheat[1][1]) )
        
        time_save = costs_no_cheating[cheat[1]] - (costs_no_cheating[start_cheat] + 2)
        if time_save >= 100:
            time_saves.append(time_save)
    # print(pd.DataFrame({"data": time_saves})["data"].value_counts().sort_index())

    # good_cheats = []
    # cheat_times = []
    # no_cheating = path_finding(None)
    # for cheat in tqdm(cheat_blocks):
    #     cheat_time = path_finding(cheat)
    #     time_save = no_cheating - cheat_time
    #     if time_save > 0:
    #         cheat_times.append(time_save)
    #         # print(no_cheating - cheat_time, cheat)
    #     if no_cheating - cheat_time >= 100:
    #         good_cheats.append(cheat)

    # print(pd.DataFrame({"data": cheat_times})["data"].value_counts().sort_index())

    return len(time_saves)



sol = solution(test_data, verbose=True)
print(sol)


100%|██████████| 86/86 [00:00<00:00, 86026.75it/s]

0


In [58]:
sol = solution(full_data)
print(sol)

100%|██████████| 13893/13893 [00:00<00:00, 356439.64it/s]

1372


# Part 2

In [ ]:

def solution2 (data, verbose=False):
    start = tuple(np.argwhere(data == 'S')[0])
    end = tuple(np.argwhere(data == 'E')[0])

    def isValid(shape : tuple, index : tuple):
            return (0 <= index[0] < shape[0] and
                    0 <= index[1] < shape[1])

    def tuple_add (t1, t2):
        return tuple(map(sum, zip(t1, t2)))
    
    # Modified for cheating
    def get_neightbors (array, pos, cheat):
        if not cheat:
            cheat = ((-1,-1), (-1,-1))
        if pos == cheat[0]:
            return [cheat[1]]

        directions = [(1,0), (-1,0), (0,1), (0,-1)]

        next_nodes = [tuple_add(pos, direction) for direction in directions
                                                                if isValid(array.shape, tuple_add(pos, direction)) 
                                                                and (array[tuple_add(pos, direction)] in ['.', 'E'] 
                                                                     or tuple_add(pos, direction) == cheat[0])
                                                                ]
        return next_nodes
    
    # To detect if node is a possible cheat node
    def get_cheat_neightbors (array, pos):
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        next_nodes = [(tuple_add(pos, direction), tuple_add(pos, tuple_add(direction, direction))) for direction in directions
                                                                if isValid(array.shape, tuple_add(pos, direction))
                                                                and  isValid(array.shape, tuple_add(pos, tuple_add(direction, direction)))
                                                                and array[tuple_add(pos, direction)] in ["#"]
                                                                and array[tuple_add(pos, tuple_add(direction, direction))] in [".", "E"] ]
        return next_nodes

    def display_field(field):
        field_list = field.tolist()
        field_str = "\n".join(["".join(row)   for row in field_list])
        return field_str
    
    
    def path_finding(cheat_block):

        # Dijkstra
        field = data.copy()

        # visited nodes -> cannot be in quere anymore
        visited = []
        # nodes to iterate over
        queue = [start]
        # Tracks lowest costs per node
        costs = defaultdict(lambda: float('inf'))
        costs[start] = 0

        while queue:
            # find smallest
            current_node = queue[0]
            # print(current_node)
            lowest_cost = costs[current_node]
            if len(queue) > 1:
                for node in queue[1:]:
                    if costs[node] < lowest_cost:
                        current_node = node
                        lowest_cost = costs[node]

            queue.remove(current_node)
            visited.append(current_node)

            nbs = get_neightbors(field, current_node, cheat_block)
            # print("Neighbors", nbs)
            for nb in nbs:
                if nb not in visited:
                    if costs[nb] > costs[current_node] +1:
                        costs[nb] = costs[current_node] +1
                        queue.append(nb)

        return costs[end], costs
    
    cheat_blocks = set()
    for start_cheat in np.argwhere(np.isin(data,  ["S", "."])):
        nbs = get_cheat_neightbors(data, start_cheat)
        cheat_blocks.update(set(nbs))


    no_cheating, costs_no_cheating = path_finding(None)

    def generateManhattanPoints(pos, ra):
        x = pos[0]
        y = pos[1]
        points = []
        for r in range(1, ra+1):
            for offset in range(r):
                invOffset = r - offset # Inverse offset
                points.append(((x + offset, y + invOffset), r))
                points.append(((x + invOffset, y - offset), r))
                points.append(((x - offset, y - invOffset), r))
                points.append(((x - invOffset, y + offset), r))
        return points


    # cheat start and end points
    start_points = set([tuple(point) for point in np.argwhere(np.isin(data,  ["S", "."]))])
    end_points = set([tuple(point) for point in np.argwhere(np.isin(data,  ["E", "."]))])

    DISTANCE = 20
    time_saves = []
    for start_point in start_points:
        cost_start = costs_no_cheating[start_point]
        possible_ends = generateManhattanPoints(start_point, DISTANCE)
        for end_point in possible_ends:
            end_point_pos = end_point[0]
            end_point_dist = end_point[1]
            if end_point_pos in end_points:
                cost_end = costs_no_cheating[end_point_pos]
                time_save = cost_end - (cost_start + end_point_dist)
                if time_save >= 100:
                    time_saves.append(time_save)

    # print(pd.DataFrame({"data": time_saves})["data"].value_counts().sort_index())

    return len(time_saves)



sol = solution2(test_data, verbose=True)
print(sol)


0


In [71]:
sol = solution2(full_data)
print(sol)

979014
